# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
import requests
import json
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import sqlite3

In [3]:
import pymongo

In [4]:
api_key = "******"

In [5]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    def __init__(self):
        self.secret_key = api_key
        self.berlin_lat = "52.5200"
        self.berlin_long = "13.4050"
        self.url_base = "https://api.darksky.net/forecast"
        self.exclude = "currently,minutely,hourly,alerts,flags"
    
    def date_convert(self, date_str):
        year = date_str[:4]
        month = date_str[5:7] 
        day = date_str[8:]
        return year, month, day
    
    def get_weather_data(self, date_str):
        year, month, day = self.date_convert(date_str)
        time = "{}-{}-{}T00:00:00".format(year, month, day) 
        url_full = "{}/{}/{},{},{}?exclude={}".format(self.url_base, self.secret_key,
                                                      self.berlin_lat, self.berlin_long,
                                                     time, self.exclude)
        response = requests.get(url_full)
        if response.status_code != 200:
            raise ValueError("API Error Response Code {}").format(response.status_code)
        else:
            return response
        
    def check_rain(self, date_str):
        response = self.get_weather_data(date_str)
        data = response.json()['daily']['data'][0]
        if data['icon'] == 'rain':
            return True
        else:
            return False
        
    def check_rain_for_all(self, date_list):
        weather = {}
        for date in date_list:
            weather[date] = self.check_rain(date)
        return weather
        

In [6]:
wg = WeatherGetter()
wg.check_rain_for_all(["2019-07-30","2019-07-31"])

{'2019-07-30': True, '2019-07-31': True}

In [7]:
conn = sqlite3.Connection('database.sqlite')
c = conn.cursor()

In [8]:
c.execute("""
          SELECT * 
          FROM Matches
          WHERE Season=2011;
          """)
matches = pd.DataFrame(c.fetchall())
matches.columns = [x[0] for x in c.description]
matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [9]:
dates = matches['Date'].unique()
dates[:5]

array(['2012-03-31', '2011-12-11', '2011-08-13', '2011-11-27',
       '2012-02-18'], dtype=object)

In [10]:
len(dates)

165

In [11]:
weather = wg.check_rain_for_all(dates)

In [12]:
weather

{'2012-03-31': True,
 '2011-12-11': False,
 '2011-08-13': False,
 '2011-11-27': False,
 '2012-02-18': False,
 '2012-01-20': True,
 '2012-02-04': False,
 '2012-04-21': False,
 '2011-09-18': True,
 '2011-10-23': True,
 '2011-10-01': False,
 '2012-03-03': False,
 '2011-08-27': True,
 '2012-03-17': False,
 '2011-11-06': True,
 '2012-05-05': True,
 '2012-04-11': False,
 '2011-12-17': True,
 '2012-02-03': False,
 '2011-10-29': False,
 '2012-01-22': True,
 '2011-12-03': True,
 '2012-04-14': False,
 '2012-03-25': False,
 '2012-03-10': False,
 '2012-04-07': False,
 '2011-11-19': False,
 '2011-10-14': False,
 '2011-09-24': False,
 '2012-04-28': False,
 '2011-12-18': True,
 '2012-03-02': False,
 '2012-03-16': False,
 '2012-02-17': True,
 '2011-08-06': True,
 '2011-11-04': False,
 '2011-09-16': False,
 '2011-07-15': False,
 '2012-05-06': True,
 '2012-02-11': False,
 '2011-10-02': False,
 '2012-03-30': False,
 '2011-10-22': False,
 '2011-08-26': True,
 '2011-08-07': True,
 '2012-02-24': True,
 '201

In [13]:
weather_df = pd.DataFrame.from_dict(weather, orient='index', columns=['Rain']).reset_index().rename(columns={"index":"Date","Rain":"Rain"})
weather_df.head()

,Date,Rain
0,2012-03-31,True
1,2011-12-11,False
2,2011-08-13,False
3,2011-11-27,False
4,2012-02-18,False


In [14]:
df = matches.merge(weather_df,how='left', on="Date")
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Rain
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,True
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,False
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,False
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,False
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,False


In [15]:
import numpy as np

In [16]:
home_teams = np.sort(matches['HomeTeam'].unique())
away_teams = np.sort(matches['AwayTeam'].unique())

In [17]:
len(home_teams)

56

In [18]:
home_teams == away_teams

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [19]:
teams = home_teams
teams_data = {}
for team in teams:
    data = {'total_matches': 0,
            'total_wins': 0,
            'total_losses': 0,
            'rain_wins': 0,
            'rain_losses': 0,
            'total_goals': 0}
    teams_data[team] = data

In [20]:
def update_team_record(name, goals, win, rain):
    team = teams_data[name]
    team['total_matches'] += 1
    team['total_goals'] += goals
    
    if win == True:
        team['total_wins'] += 1
        if rain == True:
            team['rain_wins'] += 1
    elif win == False:
        team['total_losses'] += 1
        if rain == True:
            team['rain_losses'] += 1

In [21]:
def update_data(row):
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    home_goals = row['FTHG']
    away_goals = row['FTAG']
    did_rain = row['Rain']
    
    if row['FTR'] == 'A':
        home_win = False
        away_win = True
    elif row['FTR'] == 'H':
        home_win = True
        away_win = False
    elif row['FTR'] == 'D':
        home_win = False
        away_win = False
        
    update_team_record(home_team, home_goals, home_win, did_rain)
    update_team_record(away_team, away_goals, away_win, did_rain)

In [22]:
df.apply(update_data, axis=1)
teams_data

{'Aachen': {'total_matches': 34,
  'total_wins': 6,
  'total_losses': 28,
  'rain_wins': 3,
  'rain_losses': 10,
  'total_goals': 30},
 'Arsenal': {'total_matches': 38,
  'total_wins': 21,
  'total_losses': 17,
  'rain_wins': 8,
  'rain_losses': 6,
  'total_goals': 74},
 'Aston Villa': {'total_matches': 38,
  'total_wins': 7,
  'total_losses': 31,
  'rain_wins': 2,
  'rain_losses': 10,
  'total_goals': 37},
 'Augsburg': {'total_matches': 34,
  'total_wins': 8,
  'total_losses': 26,
  'rain_wins': 4,
  'rain_losses': 6,
  'total_goals': 36},
 'Bayern Munich': {'total_matches': 34,
  'total_wins': 23,
  'total_losses': 11,
  'rain_wins': 8,
  'rain_losses': 3,
  'total_goals': 77},
 'Blackburn': {'total_matches': 38,
  'total_wins': 8,
  'total_losses': 30,
  'rain_wins': 1,
  'rain_losses': 9,
  'total_goals': 48},
 'Bochum': {'total_matches': 34,
  'total_wins': 10,
  'total_losses': 24,
  'rain_wins': 3,
  'rain_losses': 8,
  'total_goals': 41},
 'Bolton': {'total_matches': 38,
  'tot

In [23]:
for team in teams_data:
    data = teams_data[team]
    data['rain_win_pct'] = data['rain_wins'] / (data['rain_wins']+ data['rain_losses'])

In [24]:
class MongoHandler(object):
    def __init__(self):
        self.client = pymongo.MongoClient('mongodb://localhost:27017')
        self.db = self.client['section16_lab_db']
        self.collection = self.db['section16_lab_collection']
        
    def format_data(self, team_name, total_goals, total_wins, rain_win_pct):
        record = {"team_name": team_name,
                  "total_goals": total_goals,
                  "total_wins": total_wins,
                  "rain_win_pct": rain_win_pct}
        return record
    
    def update_data(self, record):
        return self.collection.insert_one(record)

In [25]:
db = MongoHandler()

for team_name, data in teams_data.items():
    record = db.format_data(team_name, data['total_goals'], data['total_wins'], data['rain_win_pct'])
    db.update_data(record)

In [26]:
[i for i in db.collection.find()]

[{'_id': ObjectId('5d4624ad6d5dce4418af5b26'),
  'team_name': 'Aachen',
  'total_goals': 180,
  'total_wins': 36,
  'rain_win_pct': 0.23076923076923078},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b27'),
  'team_name': 'Arsenal',
  'total_goals': 444,
  'total_wins': 126,
  'rain_win_pct': 0.5714285714285714},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b28'),
  'team_name': 'Aston Villa',
  'total_goals': 222,
  'total_wins': 42,
  'rain_win_pct': 0.16666666666666666},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b29'),
  'team_name': 'Augsburg',
  'total_goals': 216,
  'total_wins': 48,
  'rain_win_pct': 0.4},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b2a'),
  'team_name': 'Bayern Munich',
  'total_goals': 462,
  'total_wins': 138,
  'rain_win_pct': 0.7272727272727273},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b2b'),
  'team_name': 'Blackburn',
  'total_goals': 288,
  'total_wins': 48,
  'rain_win_pct': 0.1},
 {'_id': ObjectId('5d4624ae6d5dce4418af5b2c'),
  'team_name': 'Bochum',
  'total_goals': 2

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!

In [27]:
# c.execute("""
#           SELECT Unique_Team_ID, 
#                  TeamName,
#                  COUNT(DISTINCT Match_ID) as Matches,
#                  SUM(FTHG)+SUM(FTAG) as Goals,
#                  SUM(CASE WHEN FTR_H == 'H' THEN 1 ELSE 0 END)+ 
#                  SUM(CASE WHEN FTR_A == 'A' THEN 1 ELSE 0 END) as Wins,
#                  SUM(CASE WHEN FTR_H == 'D' THEN 1 ELSE 0 END)+ 
#                  SUM(CASE WHEN FTR_A == 'D' THEN 1 ELSE 0 END) as Draws,
#                  SUM(CASE WHEN FTR_H == 'A' THEN 1 ELSE 0 END)+ 
#                  SUM(CASE WHEN FTR_A == 'H' THEN 1 ELSE 0 END) as Losses
                 
#           FROM
#               (SELECT t.*, m1.FTHG, m1.FTR as FTR_H, m2.FTAG, m2.FTR as FTR_A
#               FROM (SELECT a.*, b.TeamName, c.Season
#                       FROM Teams_in_Matches as a
#                       LEFT JOIN Unique_Teams as b
#                       ON a.Unique_Team_ID = b.Unique_Team_ID
#                       LEFT JOIN Matches as c
#                       ON a.Match_ID = c.Match_ID) as T

#               LEFT JOIN Matches as m1
#               ON t.Match_ID = m1.Match_ID AND
#                  t.TeamName = m1.HomeTeam

#               LEFT JOIN Matches as m2 
#               ON t.Match_ID = m2.Match_ID AND
#                  t.TeamName = m2.AwayTeam

#               WHERE t.Season == 2011)
              
#           GROUP BY Unique_Team_ID, TeamName;
#           """)

# df = pd.DataFrame(c.fetchall())
# df.columns = [x[0] for x in c.description]
# df.head()